In [ ]:
import pandas as pd
import plotly.express as px
data = pd.read_csv(r'D:\my projects\segmentataion\myvenv\Online Shop Customer Sales Data.csv')

data['Purchase_DATE'] = pd.to_datetime(data['Purchase_DATE'], format="%d.%m.%y")

data.drop_duplicates(inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)
data.head(2)




In [3]:
# customer segmentation
segmented = data.groupby('Customer_id').agg({
   'Purchase_VALUE' : 'sum',
   'Revenue_Total' : 'sum',
   'N_Purchases' : 'sum',
   'Time_Spent' : 'sum'
}).reset_index()


In [ ]:
# purchase behaviour analysis
# creating a Month column to analyze purchase behaviour of customers with time
data['month'] = pd.DatetimeIndex(data['Purchase_DATE']).month

monthly_data = data.groupby( 'month')['N_Purchases'].sum().reset_index()   



#px.line(monthly_data, x='month', y='N_Purchases', title='Monthly Purchase Behaviour')
# monthly purchase behaviour by age
# categorizing the age into groups

data['age_category'] = pd.cut(data['Age'], bins=[0, 20, 40, 60, 100], labels=['0-20', '20-40', '40-60', '60+'])

age_monthly_data = data.groupby(['month', 'age_category'])['N_Purchases'].sum().reset_index()

px.bar(age_monthly_data, x='month', y='N_Purchases', color='age_category', title='Monthly Purchase Behaviour')

# purchase behaviour by gender

gender_monthly_data = data.groupby(['month', 'Gender'])['N_Purchases'].sum().reset_index()

px.line(gender_monthly_data, x='month', y='N_Purchases', color='Gender', title='Monthly Purchase Behaviour by Gender')

In [ ]:
# payment mathod analysis

# visualizing correlation between payment methods and purchase value

data['purchase_value_category'] = pd.cut(data['Purchase_VALUE'], bins=12, labels=['0-5', '5-10', '10-15', '15-20', '20-25', '25-30', '30-35', '35-40', '40-45', '45-50', '50-55', '55-60'])


payment_method_data = data.groupby(['purchase_value_category', 'Pay_Method'])['Customer_id'].count().reset_index()
payment_method_data.rename(columns={'Customer_id': 'Customer_Count'}, inplace=True)

high_value_customers = payment_method_data[(payment_method_data['purchase_value_category'] == '50-55') |
                                           (payment_method_data['purchase_value_category'] == '55-60')]

high_value_customers


fig = px.bar(payment_method_data, 
             x='purchase_value_category', 
             y='Customer_Count', 
             color='Pay_Method', 
             barmode='group',  # Ensures bars are grouped by payment method
             labels={'purchase_value_category':'Purchase Value Category', 'Customer_Count':'Number of Customers', 'Pay_Method':'Payment Method'},
             title="Number of Customers by Purchase Value Category and Payment Method")

# Show the plot
fig.show()

In [ ]:
data["time_in_mins"] = data["Time_Spent"] / 60
px.scatter(data, x='time_in_mins', y='Purchase_VALUE',  title='Time Spent vs. Purchase Value')

In [ ]:

data['Group'] = data['Newsletter'].astype(str) + ' & ' + data['Voucher'].astype(str)
px.histogram(data, x='purchase_value_category', color='Group',  title='Time Spent vs. Purchase Value')




In [10]:
data.head(2)

,Customer_id,Age,Gender,Revenue_Total,N_Purchases,Purchase_DATE,Purchase_VALUE,Pay_Method,Time_Spent,Browser,Newsletter,Voucher,month,age_category,purchase_value_category,time_in_mins,Group
0,504308,53,0,45.3,2,2021-06-22,24.915,1,885,0,0,0,6,40-60,20-25,14.750000,0 & 0
1,504309,18,1,36.2,3,2021-12-10,2.896,2,656,0,0,1,12,0-20,0-5,10.933333,0 & 1
